In [1]:
!pip install stable-baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [2]:
!pip install gymnasium

In [3]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Box, Discrete, MultiDiscrete
from collections import defaultdict
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
import random

/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/usr/local/lib/python3.11/dist-packages/shimmy/registration.py", line 304, in register_gymnasium_envs
    _register_atari_envs()
  File "/usr/local/lib/python3.11/dist-packages/shimmy/registration.py", line 205, in _register_atari_envs
    import ale_py
  File "/usr/local/lib/python3.11/dist-packages/ale_py/__init__.py", line 68, in <module>
    register_v0_v4_envs()
  File "/usr/local/lib/python3.11/dist-packages/ale_py/registration.py", line 178, in register_v0_v4_envs
    _register_rom_configs(legacy_games, obs_types, versions)
  File "/usr/local/lib/python3.11/dist-packages/ale_py/registration.py", line 63, in _register_rom_configs
    gymnasium.register(
    ^^^^^^^^^^^

In [4]:
def parse_junction_data(df):
    junctions = df['kavsak_adi'].unique()
    junction_road_mapping = {}
    for junction in junctions:
        roads = df[df['kavsak_adi'] == junction]['yol_ismi'].unique().tolist()
        junction_road_mapping[junction] = roads
    return junction_road_mapping

In [5]:
df = pd.read_csv('/kaggle/input/kavsakdata/trafik_kayit_final_multi.csv')
junction_road_mapping = parse_junction_data(df)

In [6]:
def flatten_action(multi_action, action_sizes):
    index = 0
    for i, a in enumerate(multi_action):
        factor = np.prod(action_sizes[i+1:]) if i+1 < len(action_sizes) else 1
        index += a * factor
    return index

def unflatten_action(flat_action, action_sizes):
    multi_action = []
    for size in reversed(action_sizes):
        multi_action.insert(0, flat_action % size)
        flat_action //= size
    return multi_action

In [7]:
def calculate_reward_multi(state, green_actions, prev_green_actions, waiting_counts, junction_road_mapping):
    """
    state: (flattened) tüm yolların durumu vektörü
    green_actions: her junction için hangi yol(ların) seçildiği (index listesi)
    prev_green_actions: her junction için bir önceki seçilen yol(lar) (index listesi)
    waiting_counts: her junction için bekleme sayıları (list of lists)
    junction_road_mapping: dict, junction -> road list
    """
    reward = 0
    idx = 0
    for junc_idx, (junction, roads) in enumerate(self.junction_road_mapping.items()):
        selected_set_idx = multi_action[junc_idx]
        selected_roads = self.green_sets[junction][selected_set_idx]

        selected_indices = [idx + roads.index(road) for road in selected_roads]
        selected_density = np.mean([self.state[i] for i in selected_indices])
        junction_reward = selected_density

        if selected_density < 1.0:
            junction_reward -= 2.0

        prev_green = self.prev_green_actions[junc_idx]
        if prev_green is not None and prev_green != selected_set_idx:
            junction_reward -= 0.5
        else:
            junction_reward += 0.1

        long_wait_penalty = sum(1 for w in self.waiting_counts[junc_idx] if w > 8)
        junction_reward -= 0.2 * long_wait_penalty

        reward += junction_reward

        stuck_penalty = sum(1 for val in self.state if val >= 3.8)
        reward -= 0.1 * stuck_penalty

        idx += len(roads)

    return reward / self.num_junctions  # ✅ Normalize total reward


In [8]:
def create_env_class(junction_road_mapping, green_sets, df):
    from gymnasium import Env
    from gymnasium.spaces import Discrete, Box
    import numpy as np

    class MultiJunctionTrafficEnv(Env):
        def __init__(self):
            self.junction_road_mapping = junction_road_mapping
            self.green_sets = green_sets
            self.df = df.reset_index(drop=True)
            self.current_idx = 0  # dataset içindeki index

            self.num_junctions = len(junction_road_mapping)
            self.action_sizes = [len(green_sets[junc]) for junc in junction_road_mapping.keys()]
            self.total_actions = np.prod(self.action_sizes)

            self.observation_space = Box(low=0, high=4, shape=(sum(len(roads) for roads in junction_road_mapping.values()),), dtype=np.float32)
            self.action_space = Discrete(self.total_actions)

            self.state = np.random.uniform(0, 4, size=(self.observation_space.shape[0],)).astype(np.float32)
            self.prev_green_actions = [None] * self.num_junctions
            self.waiting_counts = [[0 for _ in range(len(roads))] for roads in junction_road_mapping.values()]

            self.max_density = 4.0

        def reset(self, seed=None, options=None):
            super().reset(seed=seed)
            self.current_idx = 0
            self._update_state_from_data()
            self.prev_green_actions = [None] * self.num_junctions
            self.waiting_counts = [[0 for _ in range(len(roads))] for roads in self.junction_road_mapping.values()]
            return self.state, {}

        def _update_state_from_data(self):
            """
            Dataset'ten current_idx'teki değerlerle state günceller.
            """
            idx = 0
            for junc_idx, (junction, roads) in enumerate(self.junction_road_mapping.items()):
                for road in roads:
                    road_df = self.df[
                        (self.df['kavsak_adi'] == junction) &
                        (self.df['yol_ismi'] == road)
                    ]
                    index_in_road_df = self.current_idx % len(road_df)
                    row = road_df.iloc[index_in_road_df]

                    son_rengi = row['son_rengi']
                    self.state[idx] = son_rengi
                    idx += 1

        def _calculate_reward(self, multi_action):
            reward = 0
            idx = 0
            for junc_idx, (junction, roads) in enumerate(self.junction_road_mapping.items()):
                selected_set_idx = multi_action[junc_idx]
                selected_roads = self.green_sets[junction][selected_set_idx]

                selected_indices = [idx + roads.index(road) for road in selected_roads]
                selected_density = np.mean([self.state[i] for i in selected_indices])
                junction_reward = selected_density

                if selected_density < 1.0:
                    junction_reward -= 2.0

                prev_green = self.prev_green_actions[junc_idx]
                if prev_green is not None and prev_green != selected_set_idx:
                    junction_reward -= 0.5
                else:
                    junction_reward += 0.1

                long_wait_penalty = sum(1 for w in self.waiting_counts[junc_idx] if w > 8)
                junction_reward -= 0.2 * long_wait_penalty

                reward += junction_reward
                idx += len(roads)

            return reward

        def step(self, flat_action):
            multi_action = unflatten_action(flat_action, self.action_sizes)

            idx = 0
            for junc_idx, (junction, roads) in enumerate(self.junction_road_mapping.items()):
                selected_set_idx = multi_action[junc_idx]
                selected_roads = self.green_sets[junction][selected_set_idx]

                for r_idx, road in enumerate(roads):
                    if road in selected_roads:
                        self.state[idx + r_idx] = max(0.0, self.state[idx + r_idx] - np.random.uniform(0.6, 1.0))
                        self.waiting_counts[junc_idx][r_idx] = 0
                    else:
                        increase = np.random.uniform(0.05, 0.2)
                        if self.state[idx + r_idx] < 3.0:
                            increase += 0.05
                        self.state[idx + r_idx] = min(self.max_density, self.state[idx + r_idx] + increase)
                        self.waiting_counts[junc_idx][r_idx] += 1

                idx += len(roads)

            self.current_idx = (self.current_idx + 1) % len(self.df[self.df['kavsak_adi'] == list(self.junction_road_mapping.keys())[0]])
            self._update_state_from_data()

            reward = self._calculate_reward(multi_action)
            self.prev_green_actions = multi_action.copy()
            done = False
            info = {}

            return self.state, reward, False, False, info

    return MultiJunctionTrafficEnv


In [9]:
green_sets = {
    "Kavsak 1": {
        0: ["Gazi Yakup Satar Cd. (Kuzey)", "Gazi Yakup Satar Cd. (Guney)"],   # Ana kuzey-güney geçiş
        1: ["Hatboyu-2 Cd.", "Zubeyde Hanim Blv. (Sag)"],                            # Doğu-batı geçiş
        2: ["Zubeyde Hanim Blv. (Sag)", "Gazi Yakup Satar Cd. (Kuzey)"],
        3: ["Hatboyu-2 Cd.", "Gazi Yakup Satar Cd. (Guney)"] # Keskin dönüş kombinasyonu (çakışmazsa)
    },
    "Kavsak 2": {
        0: ["Sakarya-2 Cd.", "Sakarya-1 Cd."],
        1: ["Zubeyde Hanim Blv. (Sol)", "Zubeyde Hanim Blv. (Sag)"],
        2: ["Sakarya-1 Cd.", "Zubeyde Hanim Blv. (Sol)"], 
        3: ["Sakarya-2 Cd.", "Zubeyde Hanim Blv. (Sag)"]
    },
    "Kavsak 3": {
        0: ["Cengiz Topel Cd.", "Yilmaz Buyukersen Blv."],
        1: ["Zubeyde Hanim Blv. (Sol)"]
    }
}

junction_road_mapping = {
    "Kavsak 1": ["Hatboyu-2 Cd.", "Gazi Yakup Satar Cd. (Kuzey)", "Gazi Yakup Satar Cd. (Guney)", "Zubeyde Hanim Blv. (Sag)"],
    "Kavsak 2": ["Sakarya-2 Cd.", "Zubeyde Hanim Blv. (Sag)", "Zubeyde Hanim Blv. (Sol)", "Sakarya-1 Cd."],
    "Kavsak 3": ["Cengiz Topel Cd.", "Zubeyde Hanim Blv. (Sol)", "Yilmaz Buyukersen Blv."]
}


In [10]:
EnvClass = create_env_class(junction_road_mapping, green_sets, df)
env = EnvClass()

obs, info = env.reset()
print("Initial state:", obs)

action = 0  # flat_action = 0 → ilk junction’da ilk yol
next_obs, reward, terminated, truncated, info = env.step(action)

print("Next state:", next_obs)
print("Reward:", reward)


Initial state: [0. 0. 1. 1. 0. 4. 0. 0. 0. 1. 0.]
Next state: [0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0.]
Reward: -5.199999999999999


In [11]:
# Env'i wrap et
monitored_env = Monitor(env)
vec_env = DummyVecEnv([lambda: monitored_env])

In [12]:
from stable_baselines3 import DQN
print(torch.cuda.is_available())
'''
model = DQN(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=5e-4,
    buffer_size=10000,
    learning_starts=1000,
    batch_size=64,
    gamma=0.95,
    exploration_fraction=0.1,
    exploration_final_eps=0.05,
    target_update_interval=250,
    train_freq=1,
    gradient_steps=1,
    device="cuda"
)
'''
model = DQN(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=1e-3,         # 💡 Slightly higher LR → faster learning
    buffer_size=5000,           # 💡 Smaller replay buffer
    learning_starts=500,        # 💡 Starts learning earlier
    batch_size=32,              # 💡 Smaller batch size
    gamma=0.95,
    exploration_fraction=0.2,   # 💡 Slightly more exploration
    exploration_final_eps=0.1,
    target_update_interval=500, # 💡 Less frequent target updates
    train_freq=4,               # 💡 Train less frequently (every 4 steps)
    gradient_steps=1,
    device="cuda"
)
model.learn(total_timesteps=10000)
obs = env.reset()[0]
for _ in range(20):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    print(f"State: {obs}, Action: {action}, Reward: {reward}")
    if done or truncated:
        obs = env.reset()[0]

True
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
State: [0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0.], Action: 0, Reward: -1.6999999999999997
State: [0. 0. 1. 1. 0. 4. 0. 0. 0. 1. 0.], Action: 0, Reward: 1.2000000000000002
State: [0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0.], Action: 0, Reward: -2.3
State: [0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0.], Action: 0, Reward: -2.3
State: [0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0.], Action: 0, Reward: -1.6999999999999997
State: [0. 0. 1. 0. 0. 1. 0. 0. 0. 2. 0.], Action: 0, Reward: -1.1999999999999997
State: [0. 0. 1. 0. 0. 1. 0. 0. 0. 2. 0.], Action: 0, Reward: -1.1999999999999997
State: [0. 0. 1. 0. 0. 1. 0. 0. 0. 2. 0.], Action: 0, Reward: -1.1999999999999997
State: [0. 0. 1. 0. 0. 1. 0. 0. 0. 2. 0.], Action: 0, Reward: -2.1999999999999997
State: [0. 0. 1. 0. 0. 1. 0. 0. 0. 2. 0.], Action: 0, Reward: -2.1999999999999997
State: [0. 0. 1. 0. 0. 1. 0. 0. 0. 2. 0.], Action: 0, Reward: -2.1999999999999997
State: [0. 0. 1. 0. 0. 1. 0. 

In [13]:
def generate_realistic_multi_sim(model, junction_road_mapping, green_sets, steps=50, print_output=True):
    import numpy as np

    total_roads = sum(len(roads) for roads in junction_road_mapping.values())
    state = np.random.uniform(0, 2.5, size=(total_roads,)).astype(np.float32)  # daha düşük başlangıç
    max_density = 4.0
    state_log = []
    prev_actions = [None] * len(junction_road_mapping)
    action_counts = [np.zeros(len(green_sets[junc])) for junc in junction_road_mapping.keys()]
    waiting_counts = [np.zeros(len(roads), dtype=int) for roads in junction_road_mapping.values()]

    action_sizes = [len(green_sets[junc]) for junc in junction_road_mapping.keys()]

    for step in range(steps):
        flat_action, _ = model.predict(state, deterministic=True)
        multi_action = unflatten_action(flat_action, action_sizes)

        idx = 0
        new_state = state.copy()

        for junc_idx, (junction, roads) in enumerate(junction_road_mapping.items()):
            selected_set_idx = multi_action[junc_idx]
            selected_roads = green_sets[junction][selected_set_idx]
            road_indices = [idx + r for r in range(len(roads))]

            selected_indices = [idx + roads.index(road) for road in selected_roads]
            action_counts[junc_idx][selected_set_idx] += 1

            # 🧠 Önceki aksiyonla karşılaştırıp engellemeyi kaldırdık
            # Çünkü sürekli önceki aksiyonu korumak, ilerlemeyi engelliyor

            for r_idx in road_indices:
                if r_idx in selected_indices:
                    new_state[r_idx] = max(0.0, new_state[r_idx] - np.random.uniform(0.6, 1.0))
                    waiting_counts[junc_idx][r_idx - idx] = 0
                else:
                    increase = np.random.uniform(0.05, 0.2)
                    if state[r_idx] < 3.0:
                        increase += 0.05
                    new_state[r_idx] = min(max_density, new_state[r_idx] + increase)
                    waiting_counts[junc_idx][r_idx - idx] += 1

            # 🔁 KULLANIM AZ ise ek yük bindirme kuralı iptal edildi
            # İstersen burada analiz yapıp uyarı bastırabilirsin ama değer değiştirme kaldırıldı

            idx += len(roads)

        state_log.append({
            "step": step,
            "state": state.tolist(),
            "action": multi_action,
            "next_state": new_state.tolist(),
            "waiting_counts": [wc.tolist() for wc in waiting_counts],
            "prev_actions": prev_actions.copy()
        })

        if print_output:
            print(f"[{step}] State: {np.round(state, 1)} → Action: {multi_action} → Next: {np.round(new_state, 1)}")

        prev_actions = multi_action.copy()
        state = new_state

    return state_log


In [14]:
log = generate_realistic_multi_sim(model, junction_road_mapping, green_sets, steps=100, print_output=True)

# İstersen CSV'ye yaz:
import pandas as pd
df = pd.DataFrame(log)
df.to_csv("model_test_simulasyon.csv", index=False)

[0] State: [0.3 1.  0.6 1.6 0.  1.3 2.4 1.  1.2 1.2 2.1] → Action: [1, 3, 0] → Next: [0.  1.1 0.8 0.7 0.  0.6 2.6 1.2 0.4 1.4 1.4]
[1] State: [0.  1.1 0.8 0.7 0.  0.6 2.6 1.2 0.4 1.4 1.4] → Action: [0, 1, 1] → Next: [0.2 0.2 0.1 0.8 0.2 0.  2.  1.4 0.6 0.6 1.6]
[2] State: [0.2 0.2 0.1 0.8 0.2 0.  2.  1.4 0.6 0.6 1.6] → Action: [1, 3, 0] → Next: [0.  0.5 0.3 0.  0.  0.  2.1 1.6 0.  0.8 0.6]
[3] State: [0.  0.5 0.3 0.  0.  0.  2.1 1.6 0.  0.8 0.6] → Action: [0, 1, 1] → Next: [0.1 0.  0.  0.2 0.1 0.  1.4 1.8 0.2 0.1 0.8]
[4] State: [0.1 0.  0.  0.2 0.1 0.  1.4 1.8 0.2 0.1 0.8] → Action: [2, 3, 0] → Next: [0.3 0.  0.2 0.  0.  0.  1.5 1.9 0.  0.3 0.1]
[5] State: [0.3 0.  0.2 0.  0.  0.  1.5 1.9 0.  0.3 0.1] → Action: [0, 2, 1] → Next: [0.4 0.  0.  0.1 0.1 0.2 0.7 1.  0.2 0.  0.3]
[6] State: [0.4 0.  0.  0.1 0.1 0.2 0.7 1.  0.2 0.  0.3] → Action: [2, 3, 0] → Next: [0.6 0.  0.2 0.  0.  0.  1.  1.1 0.  0.2 0. ]
[7] State: [0.6 0.  0.2 0.  0.  0.  1.  1.1 0.  0.2 0. ] → Action: [2, 3, 0] → Next